<a href="https://colab.research.google.com/github/eonurk/CNN-KAN/blob/main/CNN_KAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import torch.nn.functional as F
from tensorflow import keras

# NaiveFourierKANLayer definition (as previously discussed)
class NaiveFourierKANLayer(nn.Module):
    def __init__(self, inputdim, outdim, initial_gridsize, addbias=True):
        super(NaiveFourierKANLayer, self).__init__()
        self.addbias = addbias
        self.inputdim = inputdim
        self.outdim = outdim
        self.gridsize_param = nn.Parameter(torch.tensor(initial_gridsize, dtype=torch.float32)) #adjusted during training
        self.fouriercoeffs = nn.Parameter(torch.empty(2, outdim, inputdim, initial_gridsize)) #adjusted during training
        nn.init.xavier_uniform_(self.fouriercoeffs)
        if self.addbias:
            self.bias = nn.Parameter(torch.zeros(1, outdim))

    def forward(self, x): #Combines cosine/sine terms with learnable coefficents for Fourier expansion and sums them + bias
        gridsize = torch.clamp(self.gridsize_param, min=1).round().int()
        outshape = x.shape[:-1] + (self.outdim,)
        x = torch.reshape(x, (-1, self.inputdim))
        k = torch.reshape(torch.arange(1, gridsize + 1, device=x.device), (1, 1, 1, gridsize))
        xrshp = torch.reshape(x, (x.shape[0], 1, x.shape[1], 1))
        c = torch.cos(k * xrshp)
        s = torch.sin(k * xrshp)
        y = torch.sum(c * self.fouriercoeffs[0:1, :, :, :gridsize], (-2, -1))
        y += torch.sum(s * self.fouriercoeffs[1:2, :, :, :gridsize], (-2, -1))
        if self.addbias:
            y += self.bias
        y = torch.reshape(y, outshape)
        return y

# CNNFourierKAN model definition
class CNNFourierKAN(nn.Module):
    def __init__(self):
        super(CNNFourierKAN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2)
        self.fourierkan1 = NaiveFourierKANLayer(32*7*7, 128, initial_gridsize=100)
        self.fourierkan2 = NaiveFourierKANLayer(128, 10, initial_gridsize=100)

    def forward(self, x):
        x = F.selu(self.conv1(x))
        x = self.pool1(x)
        x = F.selu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fourierkan1(x)
        x = self.fourierkan2(x)
        return x

# Dataset and DataLoader setup

#Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#Convert data to PyTorch tensors
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

#Reshape data to add channel dimension
x_train = x_train.unsqueeze(1)
x_test = x_test.unsqueeze(1)

#Create data loaders
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)
subset_indices = np.random.choice(len(train_dataset), int(len(train_dataset) * 0.1), replace=False)
train_subset = Subset(train_dataset, subset_indices)
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# Model, Optimizer and Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNFourierKAN().to(device)
optimizer = optim.LBFGS(model.parameters(), lr=0.01)

# Training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        def closure():
            optimizer.zero_grad()
            output = model(data.to(device))
            loss = nn.CrossEntropyLoss()(output, target.to(device))
            loss.backward()
            return loss
        optimizer.step(closure)
        if batch_idx % 10 == 0:
            loss = closure()  # Re-run closure to get loss for printing
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Evaluation function
def evaluate(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

# Running training and evaluation
for epoch in range(0, 1):
    train(model, device, train_loader, optimizer, epoch)
evaluate(model, device, test_loader)


2025-02-15 17:33:11.016982: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/crl/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Train Epoch: 0 [0/6000 (0%)]	Loss: 2.022842
Train Epoch: 0 [640/6000 (11%)]	Loss: 2.445345
Train Epoch: 0 [1280/6000 (21%)]	Loss: 2.267748
Train Epoch: 0 [1920/6000 (32%)]	Loss: 2.345423
Train Epoch: 0 [2560/6000 (43%)]	Loss: 2.343441
Train Epoch: 0 [3200/6000 (53%)]	Loss: 2.423165
Train Epoch: 0 [3840/6000 (64%)]	Loss: 2.403017
Train Epoch: 0 [4480/6000 (74%)]	Loss: 2.416418
Train Epoch: 0 [5120/6000 (85%)]	Loss: 2.209141
Train Epoch: 0 [5760/6000 (96%)]	Loss: 2.254274

Test set: Average loss: 0.0095, Accuracy: 1005/10000 (10%)

